In [ ]:
#Kutuphanelerin yuklenmesi
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import gc


In [ ]:
#Previous application
import pandas as pd
previous_application = pd.read_csv("../input/home-credit-default-risk/previous_application.csv")

#Verideki ilk 5 gözlem
pd.set_option('display.max_columns', None) 
previous_application.head(2) 


In [ ]:
#Verideki tekil gözlemlerin sayısı
previous_application.nunique()


In [ ]:
#Verinin boyutu
previous_application.shape


In [ ]:
#Verinin hakkında bilgiler
previous_application.info()

In [ ]:
#Previous application tablosundaki kategorik değişkenlerin tutulması
cat_cols = [col for col in previous_application.columns if previous_application[col].nunique() < 30]
print("kategorik degisken sayısı : " , len(cat_cols))

#Verinin kategorik değişkenlerinin hedef değişkene gore durumları
def cat_summary(dataframe,target, noc=30):
    print("CATEGORICAL FEATURE ANALYSIS",end="\n\n")
    var_count = 0
    vars_more_classes = []
    for var in cat_cols:
        if dataframe[var].nunique() <= noc:  # sınıf sayısına göre seç
            print(var, ": has",dataframe[var].nunique(), "unique category","\t-",str(dataframe[var].dtypes),end="\n\n")
            print(pd.DataFrame({var: dataframe[var].value_counts(),
                                "Count": len(dataframe[var]),
                                "Ratio": 100 * dataframe[var].value_counts() / len(dataframe),
                                "TARGET_MEAN": dataframe.groupby(var)[target].mean()}),end="\n\n\n")
            var_count += 1
            
            print("\n\n")
        else:
            vars_more_classes.append(dataframe[var].name)
    print('%d categorical variables have been described' % var_count, end="\n\n")
    print('There are', len(vars_more_classes), "variables have more than", noc, "classes", end="\n\n")
    print('Variable names have more than %d classes:' % noc, end="\n\n")
    print(vars_more_classes)

In [ ]:
#Histogram cizdirilmesi icin sayısal degiskenlerin secilmesi

num_cols = [col for col in previous_application.columns if previous_application[col].dtypes != 'O' and col not in "Id"
           and previous_application[col].nunique() > 30]
print('Sayısal değişken sayısı: ', len(num_cols))


#Sayısal degiskenlerin histogramına bakılmasını sağlayan fonksiyon.
def hist_for_nums(data, numeric_cols):
    col_counter = 0
    data = data.copy()
    for col in numeric_cols:
        data[col].hist(bins=20)
        plt.xlabel(col)
        plt.title(col)
        plt.show()
        col_counter += 1
    print(col_counter, "variables have been plotted")


hist_for_nums(previous_application, num_cols)

In [ ]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = False):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns


In [ ]:

# Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows = None, nan_as_category = False):
    # Read data and merge
    df = pd.read_csv('../input/home-credit-default-risk/application_train.csv', nrows= num_rows)
    test_df = pd.read_csv('../input/home-credit-default-risk/application_test.csv', nrows= num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    

    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    
    #yeniler
    
    df["app_1"] = df["EXT_SOURCE_1"] * df["OBS_30_CNT_SOCIAL_CIRCLE"]
    df["app_2"] = df["EXT_SOURCE_1"] * df["AMT_INCOME_TOTAL"]
    df["app_3"] = df["NONLIVINGAPARTMENTS_AVG"] * df["OWN_CAR_AGE"]
    df["app_4"] = df["EXT_SOURCE_2"] * df["OBS_30_CNT_SOCIAL_CIRCLE"]
    df["app_5"] = df["EXT_SOURCE_3"] * df["OBS_30_CNT_SOCIAL_CIRCLE"]
    df["app_6"] = (df["AMT_CREDIT"] * df["AMT_ANNUITY"]) * (df["AMT_INCOME_TOTAL"])
    
    del test_df
    gc.collect()
    return df

In [ ]:

def previous_applications(num_rows = None, nan_as_category = False):
    prev = pd.read_csv('../input/home-credit-default-risk/previous_application.csv', nrows = num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= False)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    
    prev["new_1"] = (prev.AMT_DOWN_PAYMENT * prev.RATE_DOWN_PAYMENT)
    prev["new_2"] = (prev.AMT_DOWN_PAYMENT * prev.AMT_CREDIT)
    prev["new_3"] = (prev.AMT_APPLICATION *  prev.AMT_GOODS_PRICE)
    prev["new_4"] = (prev.AMT_DOWN_PAYMENT * prev.AMT_APPLICATION)
    prev["new_5"] = (prev.AMT_DOWN_PAYMENT * prev.AMT_ANNUITY)
   
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['mean'],
        'AMT_APPLICATION': ['mean'],
        'AMT_CREDIT': ['mean'],
        'APP_CREDIT_PERC': ['mean'],
        'AMT_DOWN_PAYMENT': ['mean'],
        'AMT_GOODS_PRICE': ['mean'],
        'HOUR_APPR_PROCESS_START': ['mean'],
        'RATE_DOWN_PAYMENT': ['mean'],
        'DAYS_DECISION': ['max'],
        'CNT_PAYMENT': ['mean'],

    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

In [ ]:
#Tabloların birlestirilmesi

df = application_train_test()
prev = previous_applications()
print("Previous applications df shape:", prev.shape)
df = df.join(prev, how='left', on='SK_ID_CURR')
#del prev
gc.collect()

In [ ]:
#Kategorik değiskenlerin sınıfları
prev_orj = pd.read_csv("../input/home-credit-default-risk/previous_application.csv")
prev_cat_cols = [col for col in prev_orj.columns if prev_orj[col].nunique() < 30]


for i in prev_cat_cols:
    print("*" * 100)
    print(i)
    print(prev_orj[i].unique())

**------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [ ]:
#application_train ve previous_application tablolarının özetleri(iki tablodaki sayısal degiskenlerin "TARGET"e gore durumları)
def target_summary_with_nums(data, target):
    num_names = [col for col in data.columns if len(data[col].unique()) > 30
                 and data[col].dtypes != 'O']

    for var in num_names:
        print(data.groupby(target).agg({var: np.median}), end="\n\n\n")
    
target_summary_with_nums(df , "TARGET")

**------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [ ]:
#Numerik ve kategorik degiskenlerin tespiti ve tutulması
def get_categorical_and_numeric_columns(dataframe, exit_columns, number_of_unique_classes=10):
    """
    -> Kategorik ve sayısal değişkenleri belirler.

    :param dataframe: İşlem yapılacak dataframe
    :param exit_columns: Dikkate alınmayacak değişken ismi
    :param number_of_unique_classes: Değişkenlerin sınıflarının frekans sınırı
    :return: İlk değer olarak kategorik sınıfların adını, ikinci değer olarak sayısal değişkenlerin adını döndürür.

    """

    categorical_columns = [col for col in dataframe.columns
                           if len(dataframe[col].unique()) <= number_of_unique_classes]

    numeric_columns = [col for col in dataframe.columns if len(dataframe[col].unique()) > number_of_unique_classes
                       and dataframe[col].dtype != "O"
                       and col not in exit_columns]

    return categorical_columns, numeric_columns

In [ ]:
#Kategorik değişken analizi(previous_applcation - TARGET)
prev_target = pd.concat([prev , df.TARGET] , axis = 1)
prev_target.shape

#Kategorik değişken analizi
cat_cols = [col for col in prev_target.columns if prev_target[col].nunique() < 10]
print("kategorik degisken sayısı : " , len(cat_cols))

#get_categorical_and_numeric_columns(prev_target , "TARGET" )

cat_summary(prev_target , "TARGET")

In [ ]:
#Nadirlik durumunun incelenmesi
def rare_analyser(dataframe, categorical_columns, target, rare_perc):
    """
     Data frame değişkenlerinin herhangi bir sınıfı, verilen eşik değerden düşük frekansa sahipse bu değişkenleri gösterir.
    :param dataframe: İşlem yapılacak dataframe
    :param categorical_columns: Rare analizi yapılacak kategorik değişken adları
    :param target: Analizi yapılacak hedef değişken adı
    :param rare_perc: Rare için sınır değer. Altında olanlar rare kategorisine girer.
    :return:
    """
    rare_columns = [col for col in categorical_columns
                    if (dataframe[col].value_counts() / len(dataframe) < rare_perc).any(axis=None)]

    for var in rare_columns:
        print(var, " : ", len(dataframe[var].value_counts()))

        print(pd.DataFrame({"COUNT": dataframe[var].value_counts(),
                            "RATIO": dataframe[var].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(var)[target].mean(),
                            "TARGET_MEDIAN": dataframe.groupby(var)[target].median()}),
              end="\n\n\n")

    print(len(rare_columns), " adet rare sınıfa sahip değişken var.")
    
rare_analyser(prev_target ,cat_cols , "TARGET" , 0.1)

In [ ]:
#Aykırı degerlerin sınırlarının belirlenmesi
def outlier_thresholds(dataframe, variable, low_quantile=0.05, up_quantile=0.95):
    """
    -> Verilen değerin alt ve üst aykırı değerlerini hesaplar ve döndürür.
    :param dataframe: İşlem yapılacak dataframe
    :param variable: Aykırı değeri yakalanacak değişkenin adı
    :param low_quantile: Alt eşik değerin hesaplanması için bakılan quantile değeri
    :param up_quantile: Üst eşik değerin hesaplanması için bakılan quantile değeri
    :return: İlk değer olarak verilen değişkenin alt sınır değerini, ikinci değer olarak üst sınır değerini döndürür
    """
    quantile_one = dataframe[variable].quantile(low_quantile)

    quantile_three = dataframe[variable].quantile(up_quantile)

    interquantile_range = quantile_three - quantile_one

    up_limit = quantile_three + 1.5 * interquantile_range

    low_limit = quantile_one - 1.5 * interquantile_range

    return low_limit, up_limit


In [ ]:
#Aykırı degerlerin baskılanması
def replace_with_thresholds(dataframe, numeric_columns):
    """
    Baskılama yöntemi
    Silmemenin en iyi alternatifidir.
    Loc kullanıldığından dataframe içinde işlemi uygular.
    :param dataframe: İşlem yapılacak dataframe
    :param numeric_columns: Aykırı değerleri baskılanacak sayısal değişkenlerin adları
    """
    for variable in numeric_columns:
        low_limit, up_limit = outlier_thresholds(dataframe, variable)

        dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit

        dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


In [ ]:
#Baskılama işlemi için numerik degişkenlerin tespiti
num_cols = [col for col in prev_target.columns if len(prev_target[col].unique()) > 30
                 and prev_target[col].dtypes != 'O']
#Baskılama isleminin gerçeklestirilmesi
replace_with_thresholds(prev_target, num_cols )

In [ ]:
#Sayısal degiskenlerin(previous_application) TARGET'e gore durumları 
def target_summary_with_nums(data, target):
    num_names = [col for col in data.columns if len(data[col].unique()) > 30
                 and data[col].dtypes != 'O']

    for var in num_names:
        print(data.groupby(target).agg({var: np.mean}), end="\n\n\n")
    
target_summary_with_nums(prev_target , "TARGET")

In [ ]:
#NUMERİK DEGİSKENLERİN TARGET DEGİSKENİNE GORE DURUMLARI (previous_application )
import warnings
warnings.filterwarnings("ignore")

num_cols_prev = [col for col in prev_target.columns if prev_target[col].nunique() > 30 ]

len(num_cols_prev)

#Tum degiskenlerin target degiskenine gore gorsellestirilmesi
import seaborn as sns

for i in num_cols_prev:
    
    sns.catplot(x = "TARGET"  , y= i ,data = prev_target , kind="violin")
    

In [ ]:
pd.set_option('display.max_rows', None) 
prev_target.corr()

In [ ]:
df['NEW_DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']